# Random Forest

### Mastery Checkpoint 2  
CSC 466, Winter 2022  
Samay Nathani

In [2]:
from pathlib import Path
home = str(Path.home())

In [3]:
%load_ext autoreload
%autoreload 2

import random_forest

Import our dataset  
I chose the [Palantir Stock Price Dataset from Kaggle](https://www.kaggle.com/kalilurrahman/palantir-stock-data-latest-and-updated)

In [4]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from sklearn.model_selection import train_test_split

titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']
titanic_df2 = titanic_df.loc[:,features]
titanic_df2['CabinLetter'] = titanic_df2['Cabin'].str.slice(0,1)
X = titanic_df2.drop('Cabin',axis=1)#.dropna()
X['CabinLetter'] = X['CabinLetter'].fillna("?")
X['Pclass'] = X['Pclass'].astype(str)
X['SibSp'] = X['SibSp'].astype(str)
X['Parch'] = X['Parch'].astype(str)
X = X.dropna()
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,CabinLetter
0,3,male,22.0,1,0,7.2500,S,?
1,1,female,38.0,1,0,71.2833,C,C
2,3,female,26.0,0,0,7.9250,S,?
3,1,female,35.0,1,0,53.1000,S,C
4,3,male,35.0,0,0,8.0500,S,?


In [6]:
t = titanic_df.loc[X.index,'Survived']
X2 = pd.get_dummies(X)

In [7]:
X_train, X_test, t_train, t_test = train_test_split(X2, t, test_size=0.3)

In [17]:
trees = random_forest.make_trees(X_train, t_train, n_features=20)

In [20]:
trees[0].predict(X_test)

Int64Index([ 40, 620, 249,   6, 861,  23, 360, 378, 188, 553,
            ...
            438, 672,  89, 191, 816, 872, 349, 407,  51, 703],
           dtype='int64', length=214)


0

In [22]:
pred = random_forest.vote(trees, X_test)

In [23]:
sum(t_test == pred) / len(pred)

0.616822429906542

In [24]:
airbnb_df = pd.read_csv(
    f"AB_NYC_2019.csv"
)
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787.0,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1.0,9.0,2018-10-19,0.21,6.0,365.0
1,2595,Skylit Midtown Castle,2845.0,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1.0,45.0,2019-05-21,0.38,2.0,355.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632.0,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3.0,0.0,NaN,NaN,1.0,365.0
3,3831,Cozy Entire Floor of Brownstone,4869.0,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1.0,270.0,2019-07-05,4.64,1.0,194.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192.0,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10.0,9.0,2018-11-19,0.10,1.0,0.0


In [25]:
features = ["neighbourhood_group",  "room_type", "price", "number_of_reviews"]
airbnb = airbnb_df.loc[:, features]
airbnb["price"] = (airbnb["price"] // 100) * 100
airbnb["number_of_reviews"] = (airbnb["number_of_reviews"] // 100) * 100
airbnb.dropna(inplace=True)

In [26]:
X = airbnb.loc[:, ["price", "room_type", "number_of_reviews"]]
t = airbnb["neighbourhood_group"].astype("category").cat.codes
X2 = pd.get_dummies(X)
X_train, X_test, t_train, t_test = train_test_split(X2, t, test_size=0.3)
len(X2.columns)

5

In [27]:
dtr = random_forest.DecisionTreeRegressor()

In [28]:
trees = random_forest.make_trees(X_train, t_train, n_features=3)

In [29]:
pred = random_forest.vote(trees, X_test)

In [30]:
sum(t_test == pred) / len(pred)

0.421246545901136

In [ ]:
#dtr.fit(X_train, t_train)

In [42]:
#dtr._rules

[[('price', 0.0),
  ('number_of_reviews', 0.0),
  ('room_type_Shared room', 0),
  ('room_type_Entire home/apt', 0),
  1],
 [('price', 0.0),
  ('number_of_reviews', 0.0),
  ('room_type_Shared room', 0),
  ('room_type_Entire home/apt', 1),
  1],
 [('price', 0.0), ('number_of_reviews', 0.0), ('room_type_Shared room', 1), 2],
 [('price', 0.0),
  ('number_of_reviews', 300.0),
  ('room_type_Entire home/apt', 1),
  3],
 [('price', 0.0),
  ('number_of_reviews', 300.0),
  ('room_type_Entire home/apt', 0),
  2],
 [('price', 0.0),
  ('number_of_reviews', 100.0),
  ('room_type_Entire home/apt', 0),
  ('room_type_Private room', 1),
  1],
 [('price', 0.0),
  ('number_of_reviews', 100.0),
  ('room_type_Entire home/apt', 0),
  ('room_type_Private room', 0),
  2],
 [('price', 0.0),
  ('number_of_reviews', 100.0),
  ('room_type_Entire home/apt', 1),
  1],
 [('price', 0.0),
  ('number_of_reviews', 200.0),
  ('room_type_Entire home/apt', 0),
  ('room_type_Private room', 1),
  1],
 [('price', 0.0),
  ('num